# 5. yfinance를 이용한 주식 데이터 수집

### 5-1. 필수 라이브러리 Import

In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import warnings
import numpy as np
warnings.filterwarnings('ignore')

### 5-2. RSI 계산 함수

In [2]:
# Relative Strength Index 계산 함수
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


### 5-3. 시간 조정 함수

In [3]:
# 시간을 정시로 조정하는 함수 (예: 13:30 -> 13:00)
def adjust_time_to_hour(df):
    if 'Datetime' in df.columns:
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        df['Datetime'] = df['Datetime'].dt.floor('H')
        df = df.drop_duplicates(subset=['Datetime'], keep='last')
        
    return df


### 5-4. 기술적 지표 추가 함수 (1시간 간격용)

In [4]:
def add_technical_features(df):
    
    # 수익률 계산
    df['Returns'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 이동평균
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # 지수이동평균
    df['EMA_12'] = df['Close'].ewm(span=12).mean()
    df['EMA_26'] = df['Close'].ewm(span=26).mean()
    
    # MACD
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # RSI
    df['RSI'] = calculate_rsi(df['Close'])
    
    # 볼린저 밴드
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    bb_std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # 변동성
    df['Volatility_10'] = df['Returns'].rolling(window=10).std()
    df['Volatility_20'] = df['Returns'].rolling(window=20).std()
    
    # 가격 변화
    df['Price_Change'] = df['Close'] - df['Open']
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # High-Low 스프레드
    df['HL_Spread'] = df['High'] - df['Low']
    df['HL_Spread_Pct'] = (df['High'] - df['Low']) / df['Close'] * 100
    
    # 시간 특성
    df['Hour'] = df['Datetime'].dt.hour
    df['DayOfWeek'] = df['Datetime'].dt.dayofweek
    df['Month'] = df['Datetime'].dt.month
    df['Quarter'] = df['Datetime'].dt.quarter
    
    # 거래시간 여부
    df['Is_Trading_Hours'] = ((df['Hour'] >= 9) & (df['Hour'] <= 16)).astype(int)
    df['Is_Market_Open'] = ((df['Hour'] >= 9) & (df['Hour'] < 16)).astype(int)
    df['Is_Premarket'] = ((df['Hour'] >= 4) & (df['Hour'] < 9)).astype(int)
    df['Is_Aftermarket'] = ((df['Hour'] >= 16) & (df['Hour'] <= 20)).astype(int)
    df['Is_Extended_Hours'] = (df['Is_Premarket'] | df['Is_Aftermarket']).astype(int)
    
    return df


### 5-5. 기술적 지표 추가 함수 (일별 데이터용)

In [5]:
def add_technical_features_daily(df):
    """일별 데이터용 기술적 지표 추가"""
    
    # 수익률 계산
    df['Returns'] = df['Close'].pct_change()
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    
    # 이동평균
    df['SMA_5'] = df['Close'].rolling(window=5).mean()
    df['SMA_10'] = df['Close'].rolling(window=10).mean()
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # 지수이동평균
    df['EMA_12'] = df['Close'].ewm(span=12).mean()
    df['EMA_26'] = df['Close'].ewm(span=26).mean()
    
    # MACD
    df['MACD'] = df['EMA_12'] - df['EMA_26']
    df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
    df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
    
    # RSI
    df['RSI'] = calculate_rsi(df['Close'])
    
    # 볼린저 밴드
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    bb_std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
    df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
    
    # 변동성
    df['Volatility_5'] = df['Returns'].rolling(window=5).std()
    df['Volatility_10'] = df['Returns'].rolling(window=10).std()
    df['Volatility_20'] = df['Returns'].rolling(window=20).std()
    
    # 가격 변화
    df['Price_Change'] = df['Close'] - df['Open']
    df['Price_Change_Pct'] = (df['Close'] - df['Open']) / df['Open'] * 100
    
    # High-Low 스프레드
    df['HL_Spread'] = df['High'] - df['Low']
    df['HL_Spread_Pct'] = (df['High'] - df['Low']) / df['Close'] * 100
    
    # 시간 특성 (일별 데이터용)
    df['DayOfWeek'] = pd.to_datetime(df['Date']).dt.dayofweek
    df['Month'] = pd.to_datetime(df['Date']).dt.month
    df['Quarter'] = pd.to_datetime(df['Date']).dt.quarter
    df['DayOfMonth'] = pd.to_datetime(df['Date']).dt.day
    df['WeekOfYear'] = pd.to_datetime(df['Date']).dt.isocalendar().week
    
    # 거래일 특성
    df['Is_Monday'] = (df['DayOfWeek'] == 0).astype(int)
    df['Is_Friday'] = (df['DayOfWeek'] == 4).astype(int)
    df['Is_MonthEnd'] = pd.to_datetime(df['Date']).dt.is_month_end.astype(int)
    df['Is_MonthStart'] = pd.to_datetime(df['Date']).dt.is_month_start.astype(int)
    
    return df


### 5-6. 1시간 간격 주식 데이터 수집 함수

In [6]:
# 티커를 입력받아 최근 N일간의 1시간 간격 주식 데이터를 가져오는 함수
def get_hourly_stock_data(ticker, days=365, save_to_csv=True):
    
    try:
        if days > 730:
            days = 730
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        
        stock_data = yf.download(
            ticker,
            start=start_date.strftime('%Y-%m-%d'),
            end=end_date.strftime('%Y-%m-%d'),
            interval='1h',
            prepost=True,
            progress=False
        )
        
        if stock_data.empty:
            return None
        
        stock_data = stock_data.reset_index()
        
        # 컬럼명 정리
        if 'Date' in stock_data.columns:
            stock_data = stock_data.rename(columns={'Date': 'Datetime'})
        elif stock_data.columns[0] not in ['Datetime', 'Date']:
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # 멀티레벨 컬럼 처리
        if isinstance(stock_data.columns, pd.MultiIndex):
            new_columns = []
            for col in stock_data.columns:
                if isinstance(col, tuple):
                    if col[0] == 'Datetime' or 'Date' in str(col[0]):
                        new_columns.append('Datetime')
                    else:
                        new_columns.append(col[0])
                else:
                    new_columns.append(col)
            stock_data.columns = new_columns
        
        if 'Datetime' not in stock_data.columns:
            stock_data = stock_data.rename(columns={stock_data.columns[0]: 'Datetime'})
        
        # 시간 조정
        stock_data = adjust_time_to_hour(stock_data)
        
        # 기술적 지표 추가
        stock_data = add_technical_features(stock_data)
        
        # CSV 저장
        if save_to_csv:
            filename = f"{ticker}_1hour_data_{days}days.csv"
            stock_data.to_csv(filename, index=False)
        
        return stock_data
        
    except Exception as e:
        return None


### 5-9. 다중 티커 1시간 데이터 수집 함수


In [9]:
def get_multiple_tickers_hourly(tickers, days=365, save_individual=True, save_combined=True):
    """여러 티커의 1시간 간격 데이터를 한번에 수집"""
    
    all_data = {}
    
    for ticker in tickers:
        data = get_hourly_stock_data(ticker, days=days, save_to_csv=save_individual)
        
        if data is not None:
            all_data[ticker] = data
    
    # 통합 데이터 저장
    if save_combined and all_data:
        combined_data = pd.DataFrame()
        
        for ticker, data in all_data.items():
            ticker_data = data.copy()
            ticker_data['Ticker'] = ticker
            combined_data = pd.concat([combined_data, ticker_data], ignore_index=True)
        
        combined_filename = f"multiple_stocks_1hour_data_{days}days.csv"
        combined_data.to_csv(combined_filename, index=False)
    
    return all_data


### 5-12. 데이터 요약 분석 함수

In [11]:
def analyze_data_summary(data_dict):
    """수집된 데이터 요약 분석"""
    
    for ticker, data in data_dict.items():
        if data is not None:
            missing_count = data.isnull().sum().sum()
            print(f"{ticker}: {len(data):,}개 포인트, 결측치: {missing_count}개")


### 5-13. 주요 종목 1시간 데이터 수집 실행

In [12]:
# 주요 종목 설정
tickers = ['AAPL', 'AMZN', 'TSLA', 'GOOGL', 'MSFT']

# 1시간 간격 데이터 수집 (1년)
all_stock_data = get_multiple_tickers_hourly(tickers, days=365)

# 요약 분석
analyze_data_summary(all_stock_data)

print(f"데이터 수집 완료: {len(all_stock_data)}개 종목")


AAPL: 3,905개 포인트, 결측치: 217개
AMZN: 3,905개 포인트, 결측치: 217개
TSLA: 3,905개 포인트, 결측치: 217개
GOOGL: 3,905개 포인트, 결측치: 217개
MSFT: 3,905개 포인트, 결측치: 217개
데이터 수집 완료: 5개 종목


### 5-15. 수집된 데이터 확인

In [13]:
# AAPL 1시간 데이터 상세 분석
if 'AAPL' in all_stock_data:
    aapl_1h = all_stock_data['AAPL']
    
    # 데이터 미리보기
    print("AAPL 1시간 데이터 미리보기:")
    print(aapl_1h[['Datetime', 'Open', 'High', 'Low', 'Close', 'Volume']].head())
    
    # 거래시간 데이터 분석
    trading_hours = aapl_1h[aapl_1h['Is_Trading_Hours'] == 1]
    
    print(f"\n학습용 데이터 분석:")
    print(f"전체 시간 개수: {len(aapl_1h):,}개")
    print(f"거래시간 개수: {len(trading_hours):,}개")
    
    # 기술적 지표 컬럼 확인
    tech_indicators = [col for col in aapl_1h.columns if col in ['SMA_10', 'SMA_20', 'RSI', 'MACD', 'BB_Upper', 'BB_Lower']]
    print(f"기술적 지표 컬럼: {tech_indicators}")


AAPL 1시간 데이터 미리보기:
                   Datetime    Open    High     Low   Close  Volume
0 2024-06-20 08:00:00+00:00  214.74  215.26  214.20  214.83       0
1 2024-06-20 09:00:00+00:00  214.80  214.95  214.50  214.70       0
2 2024-06-20 10:00:00+00:00  214.69  214.75  214.50  214.67       0
3 2024-06-20 11:00:00+00:00  214.65  214.69  213.67  213.91       0
4 2024-06-20 12:00:00+00:00  213.90  215.30  213.37  213.94       0

학습용 데이터 분석:
전체 시간 개수: 3,905개
거래시간 개수: 1,999개
기술적 지표 컬럼: ['SMA_10', 'SMA_20', 'MACD', 'RSI', 'BB_Upper', 'BB_Lower']


### 5-16. 데이터 수집 완료 요약

In [14]:
print("주식 데이터 수집 완료!")
print(f"수집된 종목: {list(all_stock_data.keys())}")
print("\n생성된 CSV 파일:")

import os
csv_files = [f for f in os.listdir('.') if f.endswith('.csv') and any(ticker in f for ticker in tickers)]
for file in csv_files:
    print(f"- {file}")

print("\n권장사항:")
print("✅ 1시간 간격 1년 데이터 - LSTM 학습에 최적!")
print("   → 충분한 데이터 양 + 적절한 시간 해상도")


주식 데이터 수집 완료!
수집된 종목: ['AAPL', 'AMZN', 'TSLA', 'GOOGL', 'MSFT']

생성된 CSV 파일:
- AAPL_1hour_data_365days.csv
- AMZN_1hour_data_365days.csv
- GOOGL_1hour_data_365days.csv
- MSFT_1hour_data_365days.csv
- TSLA_1hour_data_365days.csv

권장사항:
✅ 1시간 간격 1년 데이터 - LSTM 학습에 최적!
   → 충분한 데이터 양 + 적절한 시간 해상도
